In [13]:
import pandas as pd
import sklearn
import nltk
import ast
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import joblib
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\alan\AppData\Roaming\nltk_data...


True

In [14]:
df = pd.read_csv(r'df_modificado.csv')

decido hacer el sistema de recomendacion con las columnas que poseen texto por lo que elimino las columnas numericas

In [15]:
df = df.drop(['crew', 'popularity','runtime','vote_average','vote_count'], axis=1)

De la columna cast tomo los primeros 3 actores y aplico modificaciones para quitar corchetes y comillas.

In [16]:
df['cast'] = df['cast'].apply(lambda x: x.split(",")[:3] if isinstance(x, str) else x)

In [17]:
df['cast'] = df['cast'].astype(str).str.replace('[', '').str.replace(']', '')

C:\Users\alan\AppData\Local\Temp\ipykernel_14268\1483182902.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['cast'] = df['cast'].astype(str).str.replace('[', '').str.replace(']', '')


In [18]:
df['cast'] = df['cast'].str.replace("'", "")

In [19]:
primer_registro = df['cast'].iloc[0]
print(primer_registro)

"Tom Hanks", " Tim Allen", " Don Rickles"


Creo una columna donde se convinen todas las palabras de la lista de columnas.

In [20]:
columnas_convinadas = ['genres','overview']
df['texto_combinado'] = df[columnas_convinadas].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)

In [21]:
celda_completa = df.loc[0, 'texto_combinado']
print(celda_completa)

['Animation', 'Comedy', 'Family'] Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.


aplico la tokenizacion, elimino los stop_words y lemmatizo el texto.

In [22]:
df['texto_conbinado_tokenizado'] = df['texto_combinado'].apply(lambda x: word_tokenize(x))

In [23]:
stop_words = set(stopwords.words('english'))
df['texto_conbinado_sin_stopwords'] = df['texto_conbinado_tokenizado'].apply(lambda x: [word for word in x if word.lower() not in stop_words])

In [24]:
lemmatizer = WordNetLemmatizer()
df['texto_lemmatizado'] = df['texto_conbinado_sin_stopwords'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

In [25]:
celda_completa = df.loc[0, 'texto_lemmatizado']
print(celda_completa)

['[', "'Animation", "'", ',', "'Comedy", "'", ',', "'Family", "'", ']', 'Led', 'Woody', ',', 'Andy', "'s", 'toy', 'live', 'happily', 'room', 'Andy', "'s", 'birthday', 'brings', 'Buzz', 'Lightyear', 'onto', 'scene', '.', 'Afraid', 'losing', 'place', 'Andy', "'s", 'heart', ',', 'Woody', 'plot', 'Buzz', '.', 'circumstance', 'separate', 'Buzz', 'Woody', 'owner', ',', 'duo', 'eventually', 'learns', 'put', 'aside', 'difference', '.']


Se crea el modelo utilizando la tecnica de Bolsa de Palabras y la similitud del coseno para encontrar similitudes entre los textos lematizados. Se guarda el modelo en un archivo utilizando joblib.

In [39]:
def crear_modelo_recomendacion(df):
    texto_lemmatizado = [' '.join(tokens) for tokens in df['texto_lemmatizado']]

    vectorizer = CountVectorizer()
    matriz_tdm = vectorizer.fit_transform(texto_lemmatizado).astype(np.float32)

    vocabulario = vectorizer.get_feature_names_out()

    similarity_matrix = cosine_similarity(matriz_tdm)

    modelo_recomendacion = {
        'df': df,
        'vectorizer': vectorizer,
        'similarity_matrix': similarity_matrix
    }

    return modelo_recomendacion


modelo_recomendacion = crear_modelo_recomendacion(df)

joblib.dump(modelo_recomendacion, 'modelo_joblib.joblib')


['modelo_joblib.joblib']

El archivo con el modelo entrenado pesa 7GB por lo que procedo a reducir el DataFrame para cumplir con los requisitos de almacenamiento en github (2gb)

In [29]:
df = df[['title', 'texto_lemmatizado']].head(3000)

In [31]:
df.to_csv(r'df_modificado_ML.csv', index=False)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   title              3000 non-null   object
 1   texto_lemmatizado  3000 non-null   object
dtypes: object(2)
memory usage: 47.0+ KB
